In [19]:
import sqlite3
import csv

In [21]:
with open("scraped_reviews_tableau_6_29.csv", encoding="utf8") as f:
    data = []
    reader = csv.reader(f)
    for row in reader:
        data.append(row)
    f.close()
data = data[1:]
len(data)

1034

In [25]:
def write_to_db(data):
    conn = sqlite3.connect("reviewdb-csv.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS brand_table('index_brand' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
                                                        'name_brand' VARCHAR(100));""")
    c.execute("""CREATE TABLE IF NOT EXISTS product_table('index_product' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                                                        'index_brand_product' INTEGER,
                                                        'asin_product' VARCHAR(30),
                                                        'name_product' VARCHAR(1000), 
                                                        'price_product' FLOAT, 
                                                        'overall_rating_product' FLOAT,
                                                        FOREIGN KEY (index_brand_product) REFERENCES brand_table (index_brand));""")
    c.execute("""CREATE TABLE IF NOT EXISTS review_table('index_review' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                                                    'index_product_review' INTEGER, 
                                                    'rating_review' INTEGER,
                                                    'reviewer_id_review' VARCHAR(100), 
                                                    'date_review' VARCHAR(100), 
                                                    'text_review' VARCHAR(2000), 
                                                    'length_review' FLOAT,
                                                    FOREIGN KEY (index_product_review) REFERENCES product_table (index_product));""")
    conn.commit()
    conn.close()
    for entry in data:

        conn = sqlite3.connect("reviewdb-csv.db")
        c = conn.cursor()

        c.execute("SELECT index_brand FROM brand_table WHERE name_brand == ? ;", (entry[0], ))
        index_brand=c.fetchall()

        if index_brand == []:
            c.execute("INSERT INTO brand_table VALUES ( ?, ? );", (None, entry[0]))
            c.execute("SELECT index_brand FROM brand_table WHERE name_brand ==  ? ;", (entry[0],))
            index_brand=c.fetchall()



        c.execute("SELECT index_product FROM product_table WHERE asin_product == ? ;", (entry[2],))
        index_product=c.fetchall()
        if index_product == []:
            c.execute("""INSERT INTO product_table ('index_brand_product',
                                                        'asin_product',
                                                        'name_product', 
                                                        'price_product', 
                                                        'overall_rating_product') VALUES (?, ?, ?, ?, ?);""",
            (index_brand[0][0], entry[2], entry[1], float(entry[3]), float(entry[4])))
            c.execute("SELECT index_product FROM product_table WHERE asin_product == ? ;", (entry[2],))
            index_product=c.fetchall()

#         c.execute("SELECT text_review FROM review_table WHERE text_review == ? ;", (entry[8],))
#         text_review=c.fetchall()
#         if text_review == []:
        c.execute("""INSERT INTO review_table ('index_product_review', 
                                                        'rating_review',
                                                        'reviewer_id_review', 
                                                        'date_review', 
                                                        'text_review', 
                                                        'length_review') VALUES (?, ?, ?, ?, ?, ?);""", 
        (index_product[0][0], int(entry[7][0]), entry[5], entry[6], entry[8], len(entry[8].split(" "))))

        conn.commit()
        conn.close()

In [30]:

write_to_db(data)

In [31]:
def select_all_new():
    conn = sqlite3.connect("reviewdb-csv.db")
    c = conn.cursor()
    c.execute("SELECT * FROM brand_table")
    output1 = c.fetchall()
    c.execute("SELECT * FROM product_table")
    output2 = c.fetchall()
    c.execute("SELECT * FROM review_table")
    output3 = c.fetchall()
    conn.close()
    print(output1)
    print(output2)
    print(output3)
select_all_new()

[(1, 'Beyond Meat'), (2, 'Tofurky'), (3, 'Quorn'), (4, 'Quorn Foods'), (5, 'Boca'), (6, 'Gardein')]
[(1, 1, '', 'Beyond Meat from PlantBased Frozen oz lb. Package, Ground Beef Substitute, 16 Ounce', 10.29, 4.6), (2, 2, 'B000O6I9YY', 'Tofurky, Deli Slices, Hickory Smoked, 5.5 oz', 3.39, 4.6), (3, 1, 'B07B6HV451', 'Beyond Meat Beyond Sausage Plant-Based Dinner Sausage Links, Brat Original 14 oz', 9.39, 4.7), (4, 1, 'B084NF3NHR', 'Beyond Meat Beyond Breakfast Sausage Plant-Based Breakfast Patties, Classic 7.4 oz', 5.49, 4.6), (5, 3, 'B000WM2WXE', 'Quorn, Meat-Free Meatballs, 10.6 oz (Frozen)', 4.99, 4.4), (6, 4, 'B000REIDY4', 'Quorn Foods Meatless Nuggets, Vegetarian, Frozen, 10.6 Oz', 4.99, 4.6), (7, 3, 'B000WLZHQ4', 'Quorn Foods Meatless Grounds, Vegetarian, Frozen, 12 Oz', 4.99, 4.5), (8, 5, 'B000WLXM02', "Boca Original Vegan Spicy Non-GMO Soy Chik'n Veggie Patties (4 Count)", 4.49, 4.6), (9, 5, 'B000O6K852', "Boca Original Vegan Non-GMO Soy Chik'n Veggie Nuggets (10 oz Pouch)", 2.95, 

In [32]:
conn = sqlite3.connect("reviewdb-csv.db")
c = conn.cursor()
c.execute("SELECT COUNT(1) FROM review_table")
output = c.fetchall()
conn.close()
output

[(1034,)]

In [29]:
def drop_all_new():
    conn = sqlite3.connect("reviewdb-csv.db")
    c = conn.cursor()
    c.execute("DROP TABLE brand_table")
    c.execute("DROP TABLE product_table")
    c.execute("DROP TABLE review_table")
    conn.close()
drop_all_new()